<a href="https://colab.research.google.com/github/PondKann/CXR-Project/blob/main/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### import

In [103]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
from torchsummary import summary
from skimage.io import imread, imsave
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [104]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from imutils import paths
import argparse

#### Load data

In [105]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [106]:
train_dir = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/train'
validation_dir = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/test'

In [107]:
train_normal_dir = os.path.join(train_dir,'NORMAL')
train_pneumonia_dir = os.path.join(train_dir,'PNEUMONIA')
validation_normal_dir = os.path.join(validation_dir,'NORMAL')
validation_pneumonia_dir = os.path.join(validation_dir,'PNEUMONIA')

#### สร้างตารางเก็บชื่อภาพ และโรค 2
ข้อมูลในไฟล์ train ทั้ง normal และ pneunomia มารวมอยู่ในไฟล์ all_train แล้วลองนำมาทำ data augment

In [139]:
train_normal_dir = os.path.join(train_dir,'NORMAL')
train_pneumonia_dir = os.path.join(train_dir,'PNEUMONIA')

In [140]:
all_train = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/all_train'

สร้าง data dict ของรูป

In [141]:
normal = []
for i in range(len(os.listdir(train_normal_dir))):
    img = os.listdir(train_normal_dir)[i]
    normal.append(img)
tnormal = pd.DataFrame({'ImageName':[normal][0],
                                       'Type1': 0 ,
                                       'Type2': 1                      })

In [142]:
pneumonia = []
for i in range(len(os.listdir(train_pneumonia_dir))):
    img = os.listdir(train_pneumonia_dir)[i]
    pneumonia.append(img)
tpneumonia = pd.DataFrame({'ImageName':[pneumonia][0],
                                       'Type1': 1 ,
                                       'Type2': 0                        })

In [143]:
data_train = pd.concat([tnormal, tpneumonia])
data_train.head(3)

,ImageName,Type1,Type2
0,IM-0006-0001.jpeg,0,1
1,IM-0003-0001.jpeg,0,1
2,IM-0001-0001.jpeg,0,1


####แบ่งข้อมูล

In [144]:
train_img = []
for ImageName in tqdm(data_train['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test2
    image_path = all_train +'/' + ImageName                       # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))
    train_img.append(img)


100%|██████████| 148/148 [00:06<00:00, 22.22it/s]


In [145]:
data = np.array(train_img, dtype="float32") / 255.0

In [150]:
labels = np.array(data_train[['Type1']], dtype="float32")           #shape (148, 1)

In [148]:
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels,2)                                   #shape (148, 2)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [151]:
labels.shape

(148, 1)

In [ ]:
data = np.array(train_img, dtype="float32") / 255.0
labels = np.array(data_train['Type'], dtype="float32")

#le = LabelEncoder()
#labels = le.fit_transform(labels)
#labels = to_categorical(labels,2)


(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, random_state=42)

In [136]:
labels = np.array(data_train[['Type1']], dtype="float32")

In [101]:
#le = LabelEncoder()
#labels = le.fit_transform(labels)
#labels = to_categorical(labels,2)


In [138]:
labels.shape

(148, 1)

In [130]:
labels = to_categorical(labels, num_classes = 2)

In [131]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, random_state=42)

In [117]:
testX.ndim

4

In [132]:
testY.shape

(37, 2)

In [133]:
testY.argmax(axis=1)

array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1])

#### Data Augmentation

In [120]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [121]:
# initialize an our data augmenter as an "empty" image data generator
aug = ImageDataGenerator()

In [122]:
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

DenseNet201

In [123]:
base_model2 = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [124]:
base_model2.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model2.output)
prediction_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")(average_pooling_layer)
model2 = tf.keras.models.Model(inputs=base_model2.input, outputs=prediction_layer)

In [125]:
model2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [134]:
# train the network

H = model2.fit(
	x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=2)

Epoch 1/2
13/13 [==============================] - ETA: 0s - loss: 0.7255 - accuracy: 0.5000

ValueError: ignored

In [127]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model2.predict(x=testX.astype("float64"), batch_size=BS)

[INFO] evaluating network...


In [129]:
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        37

    accuracy                           1.00        37
   macro avg       1.00      1.00      1.00        37
weighted avg       1.00      1.00      1.00        37



In [ ]:
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

#### Fine tune

In [ ]:
len(base_model2.layers)

In [ ]:
base_model2.trainable = True
for layer in base_model2.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

In [ ]:
model2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
#train
model2.fit_generator(train_generator, epochs=10, 
                    validation_data=valid_generator,
                    callbacks=[callback2])